In [1]:
import pandas as pd
import seaborn as sns

In [13]:
df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [14]:
Y=df['Response']
X=df
X.drop('Response',axis=1,inplace=True)

Dropping ID column

In [3]:
train.drop(columns=['ID'],axis=1,inplace=True)

In [4]:
train.head(2)

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0


In [8]:
train['is_c1']=train['City_Code']=='C1'
test['is_c1']=test['City_Code']=='C1'
train['is_c1']=train['is_c1'].replace({False:0,True:1})
test['is_c1']=test['is_c1'].replace({False:0,True:1})

In [10]:
train['is_c2']=train['City_Code']=='C2'
test['is_c2']=test['City_Code']=='C2'
train['is_c2']=train['is_c2'].replace({False:0,True:1})
test['is_c2']=test['is_c2'].replace({False:0,True:1})

In [12]:
train['is_c3']=train['City_Code']=='C3'
test['is_c3']=test['City_Code']=='C3'
train['is_c3']=train['is_c3'].replace({False:0,True:1})
test['is_c3']=test['is_c3'].replace({False:0,True:1})

In [8]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import numpy as np

In [23]:
cat_cols=['City_Code',
 'Region_Code',
 'Accomodation_Type',
 'Reco_Insurance_Type',
 'Is_Spouse',
 'Health Indicator',
 'Holding_Policy_Type',
 'Reco_Policy_Cat']

In [25]:
X_filled = X.copy()
X_filled['Health Indicator'] = X['Health Indicator'].fillna('NA')
X_filled['Holding_Policy_Type'] = X['Holding_Policy_Type'].fillna(0).astype(np.int64)

In [15]:


X_filled = X.copy()
X_filled['Health Indicator'] = X['Health Indicator'].fillna('NA')
X_filled['Holding_Policy_Type'] = X['Holding_Policy_Type'].fillna(0).astype(np.int64)


In [26]:
X_train, X_validation, y_train, y_validation = train_test_split(X_filled, Y, train_size=0.8, random_state=2021, shuffle=True, stratify=Y)

In [27]:
model = CatBoostClassifier(
    random_seed=2021,
    eval_metric='AUC',
)
model.fit(
    X_train, y_train,
    cat_features=cat_cols,
    use_best_model=True,
    eval_set=(X_validation, y_validation),
    verbose=50
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

CatBoostError: Bad value for num_feature[non_default_doc_idx=10,feature_idx=9]="14+": Cannot convert 'b'14+'' to float

<h3>Creating Dummy Variables for City Code</h3>

In [14]:
train=pd.concat([train,pd.get_dummies(train['City_Code'],drop_first=True)],axis=1)
train=train.drop(['City_Code'],axis=1)
test=pd.concat([test,pd.get_dummies(test['City_Code'],drop_first=True)],axis=1)
test=test.drop(['City_Code'],axis=1)

In [15]:
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C33,C34,C35,C36,C4,C5,C6,C7,C8,C9
0,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [130]:
test.head(1)

,ID,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,...,C33,C34,C35,C36,C4,C5,C6,C7,C8,C9
0,50883,156,Owned,Individual,30,30,No,NaN,6.0,3.0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
train['Region_Code'].value_counts()

1       96
5       75
2       73
6       73
4       73
        ..
5548     1
5671     1
5639     1
5686     1
5524     1
Name: Region_Code, Length: 5316, dtype: int64

In [132]:
train['high_rc']=train['Region_Code'].value_counts()>80
test['high_rc']=test['Region_Code'].value_counts()>80
train['mid_rc']=(train['Region_Code'].value_counts()>50) & (train['Region_Code'].value_counts()<80)
test['mid_rc']=(test['Region_Code'].value_counts()>50) & (test['Region_Code'].value_counts()<80)
train['low_rc']=(train['Region_Code'].value_counts()<50)
test['low_rc']=(test['Region_Code'].value_counts()<50)

In [133]:
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C36,C4,C5,C6,C7,C8,C9,high_rc,mid_rc,low_rc
0,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [134]:
train['high_rc'].fillna(False,inplace=True)
test['high_rc'].fillna(False,inplace=True)
train['mid_rc'].fillna(False,inplace=True)
test['mid_rc'].fillna(False,inplace=True)
train['low_rc'].fillna(False,inplace=True)
test['low_rc'].fillna(False,inplace=True)

In [135]:
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C36,C4,C5,C6,C7,C8,C9,high_rc,mid_rc,low_rc
0,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,False,False,False


In [136]:
train['high_rc']=train['high_rc'].replace({False:0,True:1})
test['high_rc']=test['high_rc'].replace({False:0,True:1})
train['mid_rc']=train['mid_rc'].replace({False:0,True:1})
test['mid_rc']=test['mid_rc'].replace({False:0,True:1})
train['low_rc']=train['low_rc'].replace({False:0,True:1})
test['low_rc']=test['low_rc'].replace({False:0,True:1})

In [137]:
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C36,C4,C5,C6,C7,C8,C9,high_rc,mid_rc,low_rc
0,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [138]:
train.drop('Region_Code',axis=1,inplace=True)
test.drop('Region_Code',axis=1,inplace=True)

In [139]:
train.head(3)

,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,...,C36,C4,C5,C6,C7,C8,C9,high_rc,mid_rc,low_rc
0,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,...,0,0,0,0,0,0,0,0,0,0
1,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,...,0,0,1,0,0,0,0,1,0,0
2,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,...,0,0,1,0,0,0,0,0,1,0


In [16]:
train['Accomodation_Type']=train['Accomodation_Type'].replace({'Rented':0,'Owned':1})
test['Accomodation_Type']=test['Accomodation_Type'].replace({'Rented':0,'Owned':1})
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C33,C34,C35,C36,C4,C5,C6,C7,C8,C9
0,3213,0,Individual,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [17]:
train['Reco_Insurance_Type']=train['Reco_Insurance_Type'].replace({'Individual':0,'Joint':1})
test['Reco_Insurance_Type']=test['Reco_Insurance_Type'].replace({'Individual':0,'Joint':1})
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C33,C34,C35,C36,C4,C5,C6,C7,C8,C9
0,3213,0,0,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [18]:
train['age_diff']=((train['Upper_Age']-test['Lower_Age'])==0)
test['age_diff']=((test['Upper_Age']-test['Lower_Age'])==0)
train['age_diff']=train['age_diff'].replace({False:0,True:1})
test['age_diff']=test['age_diff'].replace({False:0,True:1})
#train.drop(['Upper_Age'],axis=1,inplace=True)
#test.drop(['Upper_Age'],axis=1,inplace=True)
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C34,C35,C36,C4,C5,C6,C7,C8,C9,age_diff
0,3213,0,0,36,36,No,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [19]:
train['Is_Spouse']=train['Is_Spouse'].replace({'No':0,'Yes':1})
test['Is_Spouse']=test['Is_Spouse'].replace({'No':0,'Yes':1})
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C34,C35,C36,C4,C5,C6,C7,C8,C9,age_diff
0,3213,0,0,36,36,0,X1,14+,3.0,22,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train['Health Indicator'].isna().sum()

11691

In [21]:
train['Health Indicator'].fillna('X0',inplace=True)
test['Health Indicator'].fillna('X0',inplace=True)

In [22]:
train['Health Indicator']=train['Health Indicator'].replace({'X9':1,'X8':2,'X7':3,'X6':4,'X5':5,'X4':6,'X3':7,'X2':8,'X0':9,'X1':10})
test['Health Indicator']=test['Health Indicator'].replace({'X9':1,'X8':2,'X7':3,'X6':4,'X5':5,'X4':6,'X3':7,'X2':8,'X0':9,'X1':10})

In [27]:
train.head(2)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,...,C36,C4,C5,C6,C7,C8,C9,age_diff,Holding_Policy_Duration_un,is_15
0,3213,0,0,36,36,0,10,14+,3.0,22,...,0,0,0,0,0,0,0,0,14+,0
1,1117,1,1,75,22,0,8,NaN,NaN,22,...,0,0,1,0,0,0,0,0,15,1


In [25]:
train['Holding_Policy_Duration_un']=train['Holding_Policy_Duration'].fillna(15.0)
test['Holding_Policy_Duration_un']=test['Holding_Policy_Duration'].fillna(15.0)

In [26]:
train['is_15']=train['Holding_Policy_Duration_un']==15.0
test['is_15']=test['Holding_Policy_Duration_un']==15.0
train['is_15']=train['is_15'].replace({False:0,True:1})
test['is_15']=test['is_15'].replace({False:0,True:1})

In [45]:
train=train.drop(['Holding_Policy_Duration_un'],axis=1)
test=test.drop(['Holding_Policy_Duration_un'],axis=1)

In [37]:
train=pd.concat([train,pd.get_dummies(train['Health Indicator'],drop_first=True)],axis=1)
train=train.drop(['Health Indicator'],axis=1)
test=pd.concat([test,pd.get_dummies(test['Health Indicator'],drop_first=True)],axis=1)
test=test.drop(['Health Indicator'],axis=1)
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Reco_Policy_Premium,Response,is_c1,is_c2,...,22,2,3,4,5,6,7,8,9,10
0,3213,0,0,36,36,0,11628.0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [184]:
train['Holding_Policy_Duration'].isna().sum()

20251

In [185]:
train['Holding_Policy_Duration'].fillna('0',inplace=True)
test['Holding_Policy_Duration'].fillna('0',inplace=True)

In [28]:
train=pd.concat([train,pd.get_dummies(train['Holding_Policy_Duration'],drop_first=True)],axis=1)
train=train.drop(['Holding_Policy_Duration'],axis=1)
test=pd.concat([test,pd.get_dummies(test['Holding_Policy_Duration'],drop_first=True)],axis=1)
test=test.drop(['Holding_Policy_Duration'],axis=1)
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,...,14+,14.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,3213,0,0,36,36,0,10,3.0,22,11628.0,...,1,0,0,0,0,0,0,0,0,0


In [29]:
train['Holding_Policy_Type'].isna().sum()

20251

In [30]:
train['Holding_Policy_Type'].fillna('0.0',inplace=True)
test['Holding_Policy_Type'].fillna('0.0',inplace=True)

In [31]:
train['unk_policy_type']=train['Holding_Policy_Type']==0.0
test['unk_policy_type']=test['Holding_Policy_Type']==0.0
train['unk_policy_type']=train['unk_policy_type'].replace({False:0,True:1})
test['unk_policy_type']=test['unk_policy_type'].replace({False:0,True:1})

In [32]:
train=pd.concat([train,pd.get_dummies(train['Holding_Policy_Type'],drop_first=True)],axis=1)
train=train.drop(['Holding_Policy_Type'],axis=1)
test=pd.concat([test,pd.get_dummies(test['Holding_Policy_Type'],drop_first=True)],axis=1)
test=test.drop(['Holding_Policy_Type'],axis=1)
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Reco_Policy_Cat,Reco_Policy_Premium,Response,...,5.0,6.0,7.0,8.0,9.0,unk_policy_type,2.0,3.0,4.0,0.0
0,3213,0,0,36,36,0,10,22,11628.0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
train['reco_greater_15']=train['Reco_Policy_Cat']>15
test['reco_greater_15']=test['Reco_Policy_Cat']>15
train['reco_greater_15']=train['reco_greater_15'].replace({False:0,True:1})
test['reco_greater_15']=test['reco_greater_15'].replace({False:0,True:1})

In [34]:
train=pd.concat([train,pd.get_dummies(train['Reco_Policy_Cat'],drop_first=True)],axis=1)
train=train.drop(['Reco_Policy_Cat'],axis=1)
test=pd.concat([test,pd.get_dummies(test['Reco_Policy_Cat'],drop_first=True)],axis=1)
test=test.drop(['Reco_Policy_Cat'],axis=1)
train.head(1)

,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Reco_Policy_Premium,Response,is_c1,...,13,14,15,16,17,18,19,20,21,22
0,3213,0,0,36,36,0,10,11628.0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [35]:
test.head(1)

,ID,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Reco_Policy_Premium,is_c1,...,13,14,15,16,17,18,19,20,21,22
0,50883,156,1,0,30,30,0,9,11934.0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
train.to_csv('Train_pred_2.csv',index=False)
test.to_csv('Test_pred_2.csv',index=False)

In [42]:
train.columns

Index([               'Region_Code',          'Accomodation_Type',
              'Reco_Insurance_Type',                  'Upper_Age',
                        'Lower_Age',                  'Is_Spouse',
              'Reco_Policy_Premium',                   'Response',
                            'is_c1',                      'is_c2',
                            'is_c3',                        'C10',
                              'C11',                        'C12',
                              'C13',                        'C14',
                              'C15',                        'C16',
                              'C17',                        'C18',
                              'C19',                         'C2',
                              'C20',                        'C21',
                              'C22',                        'C23',
                              'C24',                        'C25',
                              'C26',                        'C